In [ ]:
import numpy as np

from .primitive import Primitive
from .collider import Collider

from raypy.utils.constants import *


## Sphäre

Eine Sphäre (oft auch Kugel genannt) beschreibt eine runde Figur oder ihre Oberfläche. Wichtig dabei ist, dass jeder Punkt auf der Oberfläche die gleiche Distanz zum Mittelpunkt besitzt.



Wird ein Lichtstrahl ausgesendet, muss berechnet werden ob und wie der Lichstrahl zur Späre steht. Hier interessiert vorallem der erste Auftreffpunkt. Es gibt dabei 3 Möglichkeiten wie der Lichtstrahl zur Späre steht.
- Er verfehlt sie.
- Er streift sie.
- Er durchquert sie.

Wird die Späre verfehlt, dann gibt es keinen Auftreffpunkt.
Wird die Späre nur gestriffen, dann gibt es **genau einen** Auftreffpunkt.
Wird die Späre durchkreuzt, dann gibt es 2 Schnittpunkte, einen Eintrittspunkt und einen Austrittspunkt. In diesem Fall interessiert uns nur der erste Auftreffpunkt, also der Eintrittspunkt.

In [ ]:


class Sphere(Primitive):
    def __init__(self, center, material, radius, max_ray_depth=5, shadow=True):
        super().__init__(center, material, max_ray_depth, shadow)
        self.collider_list += [SphereCollider(assigned_primitive=self, center=center, radius=radius)]
        self.bounded_sphere_radius = radius

    def get_uv(self, hit):
        return hit.collider.get_uv(hit)


In [ ]:


class SphereCollider(Collider):
    def __init__(self,  radius, **kwargs):
        super().__init__(**kwargs)
        self.radius = radius

    def intersect(self, origin, direction):
        b = 2 * direction.dot(origin - self.center)
        c = self.center.square_length() + origin.square_length() - 2 * self.center.dot(origin) - (self.radius * self.radius)
        disc = (b ** 2) - (4 * c)
        sq = np.sqrt(np.maximum(0, disc))
        h0 = (-b - sq) / 2
        h1 = (-b + sq) / 2
        h = np.where((h0 > 0) & (h0 < h1), h0, h1)
        pred = (disc > 0) & (h > 0)
        M = (origin + direction * h)
        NdotD = ((M - self.center) * (1. / self.radius)).dot(direction)

        pred1 = (disc > 0) & (h > 0) & (NdotD > 0)
        pred2 = (disc > 0) & (h > 0) & (NdotD < 0)
        pred3 = True

        # return an array with hit distance and the hit orientation
        return np.select([pred1, pred2, pred3],
                         [[h, np.tile(UPDOWN, h.shape)], [h, np.tile(UPWARDS, h.shape)], FARAWAY])

    def get_normal(self, hit):
        return (hit.point - self.center) / self.radius